In [12]:
import os.path
import sys
sys.path.append('/home/chenyidong/keops')
import numpy as np
import torch
from random import choices
import imageio
from pykeops.torch import LazyTensor
from pykeops.torch.cluster import grid_cluster
import pandas as pd
import numpy as np
from pykeops.torch.cluster import cluster_ranges_centroids
from pykeops.torch import generic_argmin
from pykeops.torch import generic_argmin
from pykeops.torch import generic_sum,generic_logsumexp,generic_argmin
from pykeops.torch import generic_argmin
from pykeops.torch import generic_sum,generic_logsumexp,generic_argmin
use_cuda = torch.cuda.is_available()
dtype = torch.cuda.DoubleTensor if use_cuda else torch.FloatTensor
from mytools import load_image,RGB_cloud,get_measure,combine_measure,squared_distances,sort_clusters,progate
from SparseSinkhorn import  sinkhorn_on_log_domain_GPU_keops,sinkhorn_on_log_domain_GPU
 

In [13]:
d = 3
n = 500
X_i_cpu = np.random.randn(n,d) 
Y_j_cpu = np.random.randn(n,d)
X_i = torch.from_numpy(X_i_cpu).type(dtype).view(-1, d)
Y_j = torch.from_numpy(Y_j_cpu).type(dtype).view(-1, d)
a = np.ones((len(X_i_cpu),1))/X_i_cpu.shape[0]
b = np.ones((len(Y_j_cpu),1))/Y_j_cpu.shape[0]

In [14]:
# solve by the pot !pip install POTa
import ot

M = np.sum((X_i_cpu[:, None, :] - Y_j_cpu[None, :, :]) ** 2, 2)
T = ot.emd(np.squeeze(a), np.squeeze(b), M)  # exact linear program
distance = np.sum(np.multiply(T,M))

In [15]:
# solve by the sinkhorn on log domain
reg=1e-3
f,g,regk = sinkhorn_on_log_domain_GPU(a, b, M, reg, epsilon0 = 1e6, numItermax=30,stopThr=1e-9)
def get_K(alpha, beta,reg,M):
    """log space computation"""
    device = torch.device('cuda')
    M_ = torch.as_tensor(M, dtype=torch.float64,device=device)
    return torch.exp(-(M_ - alpha - beta) / reg),M_
T_log_domain,M_ = get_K(f, g,regk,M)
distance_log = torch.sum(torch.mul(T_log_domain,M_))

tensor(1.1101e-30, device='cuda:0', dtype=torch.float64)
tensor(1., device='cuda:0', dtype=torch.float64)
tensor(6.6129e-31, device='cuda:0', dtype=torch.float64)
tensor(2., device='cuda:0', dtype=torch.float64)
tensor(1.2427e-30, device='cuda:0', dtype=torch.float64)
tensor(3., device='cuda:0', dtype=torch.float64)
tensor(5.2897e-31, device='cuda:0', dtype=torch.float64)
tensor(4., device='cuda:0', dtype=torch.float64)
tensor(1.3743e-30, device='cuda:0', dtype=torch.float64)
tensor(5., device='cuda:0', dtype=torch.float64)
tensor(5.4571e-31, device='cuda:0', dtype=torch.float64)
tensor(6., device='cuda:0', dtype=torch.float64)
tensor(6.9194e-31, device='cuda:0', dtype=torch.float64)
tensor(7., device='cuda:0', dtype=torch.float64)
tensor(1.1328e-30, device='cuda:0', dtype=torch.float64)
tensor(8., device='cuda:0', dtype=torch.float64)
tensor(5.8032e-31, device='cuda:0', dtype=torch.float64)
tensor(9., device='cuda:0', dtype=torch.float64)
tensor(1.2449e-30, device='cuda:0', dtype=torc

In [20]:
# solve by the sparse sinkhorn 

from mytools import load_image,RGB_cloud,get_measure,combine_measure,squared_distances,sort_clusters,progate

def multi_level_sinkhorn(a,b,X_i,Y_j,reg = 1e-3):
    eps = 0.1  # Size of our square bins 
    x_labels_ = grid_cluster(X_i, eps)  # class labels
    y_labels_ = grid_cluster(Y_j, eps)  # class labels
    x_ranges, x_centroids, _ = cluster_ranges_centroids(X_i, x_labels_)
    y_ranges, y_centroids, _ = cluster_ranges_centroids(Y_j, y_labels_)
    x, x_labels,orginalx = sort_clusters(X_i, x_labels_)
    y, y_labels,orginaly = sort_clusters(Y_j, y_labels_)
    measurex = torch.as_tensor(a).type(dtype)
    measurey = torch.as_tensor(b).type(dtype) 
    mux = combine_measure(x_labels,a)
    muy = combine_measure(y_labels,b)
    # solve the first layer
    reg = 1e-1
    debug = 0
    f,g,regk = sinkhorn_on_log_domain_GPU_keops(mux, muy, x_centroids, y_centroids , reg,  
                                                epsilon0 = 1e6, numItermax=30,stopThr=1e-9,
                                                debug=debug,ranges = None)


    torch.cuda.empty_cache()
    x_centroids_cpu = x_centroids.cpu()
    y_centroids_cpu = y_centroids.cpu()
    f_cpu = f.cpu()
    g_cpu = g.cpu()
    x_ranges_cpu = x_ranges.cpu()
    y_ranges_cpu = y_ranges.cpu()
    D_cpu =  -torch.sum((x_centroids_cpu[:, None, :] - y_centroids_cpu[None, :, :]) ** 2, 2) + f_cpu + g_cpu.t()



    thetalist = torch.logspace(-10,-12,steps=100)
    for theta in  thetalist:
        #print(theta)

        theta = torch.as_tensor(theta).type(dtype)
        temp = torch.log(theta)* regk
        keep = D_cpu > temp.cpu()
        ranges_ij_cpu   = from_matrix(x_ranges_cpu, y_ranges_cpu, keep)


        if torch.all(torch.sum(keep,dim=0)>0) and torch.all(torch.sum(keep,dim=1)>0):
            break


    ranges_ij  = (ranges_ij_cpu[0].cuda(),ranges_ij_cpu[1].cuda(),
                  ranges_ij_cpu[2].cuda(),ranges_ij_cpu[3].cuda(),
                  ranges_ij_cpu[4].cuda(),ranges_ij_cpu[5].cuda())

    areas = (x_ranges[:, 1] - x_ranges[:, 0])[:, None] * (y_ranges[:, 1] - y_ranges[:, 0])[
        None, :
    ]
    total_area = torch.sum(areas)  # should be equal to N*M
    sparse_area = torch.sum(areas[keep])
    print(
        "We keep {:.3e}/{:.3e} = {:.3f}% of the original kernel matrix.".format(
            sparse_area, total_area, float(100 * sparse_area / total_area)
        )
    )
    # progagation 
    fnew = progate(f,x_labels)  
    gnew = progate(g,y_labels)  
    fnew_ = LazyTensor(fnew[:, None])
    gnew_ = LazyTensor(gnew[:, None])
    

    debug = 0

    fall,gall,regk = sinkhorn_on_log_domain_GPU_keops(measurex, measurey, X_i, Y_j, reg,
                                                      epsilon0 = 1e1, numItermax=30,stopThr=1e-9,ranges = ranges_ij,
                                                       warmf=fnew,warmg=gnew, debug=debug)
    subset = ranges_ij
    return fall,gall,regk,subset



fall,gall,regk,subset = multi_level_sinkhorn(a,b,X_i,Y_j,1e-3)
distance_sparse = compute_distance(d,X_i,Y_j,fall,gall,regk,ranges_ij=subset)
print(distance_sparse)

tensor(5.5415e-16, device='cuda:0', dtype=torch.float64)
tensor(1.9282e-06, device='cuda:0', dtype=torch.float64)
tensor(4.7927e-07, device='cuda:0', dtype=torch.float64)
We keep 4.125e+04/2.500e+05 = 16.500% of the original kernel matrix.
tensor(4.0134e-06, device='cuda:0', dtype=torch.float64)
tensor(1.5817e-06, device='cuda:0', dtype=torch.float64)
tensor(8.9029e-07, device='cuda:0', dtype=torch.float64)
tensor(0.8730, device='cuda:0', dtype=torch.float64)


In [21]:
# for high dimention problem
d = 10
n = 50000
X_i_cpu = np.random.randn(n,d) 
Y_j_cpu = np.random.randn(n,d)
X_i = torch.from_numpy(X_i_cpu).type(dtype).view(-1, d)
Y_j = torch.from_numpy(Y_j_cpu).type(dtype).view(-1, d)
a = np.ones((len(X_i_cpu),1))/X_i_cpu.shape[0]
b = np.ones((len(Y_j_cpu),1))/Y_j_cpu.shape[0]


In [22]:
# solve by the pot !pip install POTa  caused memory problems
import ot

M = np.sum((X_i_cpu[:, None, :] - Y_j_cpu[None, :, :]) ** 2, 2)
T = ot.emd(np.squeeze(a), np.squeeze(b), M)  # exact linear program
distance = np.sum(np.multiply(T,M))

MemoryError: Unable to allocate 186. GiB for an array with shape (50000, 50000, 10) and data type float64

In [27]:
from SparseSinkhorn import  sinkhorn_on_log_domain_GPU_keops_high_dimension
measurex = torch.as_tensor(a).type(dtype)
measurey = torch.as_tensor(b).type(dtype) 
dimension = 10
reg = 1e-4
fall,gall,regk = sinkhorn_on_log_domain_GPU_keops_high_dimension(dimension,measurex, measurey, X_i, Y_j, reg, debug = 0,
                                                 epsilon0 = 1e6, numItermax=30,stopThr=1e-9 )

tensor(9.8635e-17, device='cuda:0', dtype=torch.float64)
tensor(0.0003, device='cuda:0', dtype=torch.float64)
tensor(6.9727e-05, device='cuda:0', dtype=torch.float64)


In [10]:
# compare the three results
print(distance_sparse)